### Ecuación de onda unidimensional
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/VladP008/NotasEDPs/blob/main/EcuacionesHiperbolicas/CuerdaAtada/CondicionDirichlet/CuerdaAtadaInterv-F.ipynb)

#### Cuerda atada en ambos extremos
##### Condición de frontera tipo Dirichlet

Recordemos el siguiente problema sobre el semieje positivo con valores iniciales y de frontera
\begin{equation}
   \left\{
      \begin{aligned}
        u_{tt} &= c^{2}u_{xx} &&0<x<l, \quad t>0, \\
        u(0,t) & =u(l,t)=0 && t>0 \\
        u(x,0) &= f(x) && x>0,\\
        u_t(x,0) &= 0 && 0<x<l,
      \end{aligned}
    \right.
\end{equation}

Donde $f$ es $\textit{el perfil inicial de la cuerda}$ y $f(0)=f(l)=0$

Sabemos que la solución está dada por 
\begin{equation*}
    \boxed{u(x, t)=\frac{\tilde{f}(x-c t)+\tilde{f}(x+c t)}{2}}
\end{equation*}

Con  $\tilde{f}$ la extensión a $(-l,0)$

\begin{equation*}
    \tilde{f}(x)= \left\{ \begin{array}{lcc}
                f(x) &   si  & 0<x<l, \\
                 \\ -f(-x) &  si & -l<x < 0. \\
                \end{array}
    \right.
\end{equation*}
Y luego se extiende a toda $\mathbb{R}$ de manera $2l-periódica$

#### Código para visualizar el fenómeno
Observaciones importantes:
* No podemos hacer la extensión de $\tilde{f}$ a toda $\mathbb{R}$, así que nos reducimos a una cantidad finita
* Lo anterior implica que solo habrá una cantidad finita de reflexiones de la onda

In [14]:
# Se utilizan las siguientes bibliotecas:
import matplotlib.pyplot as plt    # Para las gráficas
import numpy as np                 # Para el uso de arrays
import matplotlib.animation as animation    # Para la animación
from IPython.display import HTML            # Para poder visualizar la animación en el notebook


# La siguiente instrucción se utiliza para habilitar el modo interactivo en el notebook
%matplotlib notebook

# Parámetros del problema
T = 0.5                  # Tensión de la cuerda
p = 5                    # Densidad de la cuerda
c = (T/p)**(0.5)         # Velocidad de onda
l = 10                   # Longitud de la cuerda        
n = 2000                  # Número de puntos para la discretización del dominio espacial (por cada intervalo)
x0 = np.linspace(-2*l,-l, n + 1) # Puntos en el intervalo [-2l,-l]
x1 = np.linspace(-l,0, n + 1)   # Puntos en el intervalo [-l,0] 
x2 = np.linspace(0,l, n + 1)    # Puntos en el intervalo [l,0]
x3 = np.linspace(l,2*l, n + 1)  # Puntos en el intervalo [l,2l] 
x4 = np.linspace(2*l,3*l, n +1) # Puntos en el intervalo [2l,3l]
x = np.append(x0,[x1,x2])       # Dominio: [-2l,0]
x = np.append(x,[x3,x4])        # Dominio: [-2l,2l]
tmax = 57                       # Tiempo máximo de la animación
t = np.linspace(0, tmax, 50)   # Vector del tiempo


# Definición del perfil inicila de la onda (f(x))
def f(x):
    """Función que regresa el valor f(x) que corresponde a la función del
       perfil inicial

    Parametros
    ----------
    x : array
        Valores del dominio

    Returns
    -------
    y: array 
        Valores f(x)
    """
    # Inicializa el array y (mismo tamaño que x) con ceros 
    y = np.zeros_like(x)
    # Descomentar segun la función deseada
    ################################
    # Una función de soporte compacto y clase C-infinito
    # Se utilizan mascaras booleanas 
    # condicion1 = (-1+l/2<x) & (x<1+l/2)
    # y[condicion1] = np.exp(-1/(1-np.power(np.absolute(x[condicion1]-l/2),2)))
    ################################
    ## Una gaussiana (Nota: esta función no cumple la condición f(0)=f(l)=0)
    # y = np.exp(-(x-l/2)**2)
    ################################
    ## Un triángulo (Nota:esta función no es derivable en el pico)
    ## Se utilizan mascaras booleanas 
    condicion1 = (x >= -1+l/2) & (x <= 0+l/2)    # array de valores booleanos,cuya entrada i depende de si se cumplen las condiciones en la entrada i de x
    condicion2 = (x > 0+l/2) & (x <= 1+l/2)      # array de valores booleanos,cuya entrada i dependen de si se cumplen las condiciones en la entrada i de x
    y[condicion1] = x[condicion1]-l/2+1    # Se eligen los elementos que cumplen la condicion1 y se le asigna el valor x +1
    y[condicion2] = -x[condicion2]+l/2+1  # Se eligen los elementos que cumplen la condicion2 y se le asigna el valor -x +1
    ################################

    return y
# Definición del perfil inicila de la onda (\tilde{f}(x))
def F(x):
    """Función que regresa el valor F(x) que corresponde a la extensión
       impar de la función f
    

    Parametros
    ----------
    x : array
        Valores del dominio

    Returns
    -------
    z: array 
        Valores F(x)
    """
    # Inicializa el array y (mismo tamaño que x) con ceros 
    z = np.zeros_like(x)
    condicion0 = (-2*l<=x) & (x<-l)
    condicion1 = (-l<=x) & (x<0)   
    condicion2 = (0<=x) & (x<l)  
    condicion3 = (l<=x) &(x<=2*l)  
    condicion4 = (2*l<=x) & (x<=3*l)

    # Aqui definimos la extensión impar de f i.e. es la extensión en (-l,l)
    z[condicion1] = -f(-x[condicion1])
    z[condicion2] = f(x[condicion2])

    # Para trbajar con la periodiocidad en los intervalos restantes solo hacemos ajustes en el argumento para llevarlos a [-l,0] o [0,l]
    z[condicion0] = f((x[condicion0] + 2*l))
    z[condicion3] = -f(-(x[condicion3] - 2*l))
    z[condicion4] = f((x[condicion4] - 2*l))
    return z
    
# Solución analítica del problema
def u(x,t):
    """Función que regresa el valor u(x,t) que corresponde a la solución
       analítica del problema
    
    Parametros
    ----------
    x : array
        Valores del dominio
    t : array
        Valores del tiempo

    Returns
    -------
    : array 
        Solución u(x,t)
    """
    return (F(x-c*t)+F(x+c*t))/2    # Formula de d'Alembert
    
# Se crea la figura, se fijan parámetros y se definen los objetos de linea y texto
fig, ax = plt.subplots()
line0, = ax.plot(x0, u(x0, t[0]), color='blue',linestyle='dashed') # Primera imagen
line1, = ax.plot(x1, u(x1, t[0]), color='blue',linestyle='dashed') # Primera imagen
line2, = ax.plot(x2, u(x2, t[0]), color='blue') # Primera imagen
line3, = ax.plot(x3, u(x3, t[0]), color='blue',linestyle='dashed') # Primera imagen
line4,= ax.plot(x4, u(x4, t[0]), color='blue',linestyle='dashed') # Primera imagen
text = ax.text(l/2, -0.8, f'c= {t[0]}', fontsize=10, color='black', ha='center', va='center',bbox=dict(facecolor='red', alpha=0.5))  # Para mostrar el tiempo en la animación

# Ajustes de los ejes, etiquetas y título
ax.set_title('Cuerda atada en ambos extremos')
ax.grid(color='grey', linestyle='-', linewidth=0.1)
ax.set_ylim([-1.0, 1.0])      # Se fijan los límites del eje y
ax.set_xlim([0, l]) # Se fijan los límites del eje x
ax.set_xlabel('x')

# Para mostrar la variable c en una caja
ax.text(l/2, 0.8, f'c= {round(c,2)}', fontsize=10, color='black', ha='center', va='center', bbox=dict(facecolor='white', alpha=0.5))

# Función de actualización de cada frame
def update(frame):
    """Función que regresa el valor de la solución para cada frame de la animación
    
    Parametros
    ----------
    frame : int
        Indice del arreglo del tiempo

    """
    y0 = u(x0, t[frame])
    y1 = u(x1, t[frame])
    y2 = u(x2, t[frame])
    y3 = u(x3, t[frame])
    y4 = u(x4, t[frame])
    line0.set_ydata(y0)
    line1.set_ydata(y1)
    line2.set_ydata(y2)
    line3.set_ydata(y3)
    line4.set_ydata(y4)
    text.set_text(f't= {round(t[frame],1)}')

# Crear la animación
animacion = animation.FuncAnimation(fig, update, frames=len(t), interval=250)

# Mostrar la animación en el notebook
HTML(animacion.to_jshtml())

# Se guarda en formato gif
#animacion.save(filename="CuerdaAtada-Intervalo_T.gif", writer="pillow")

<IPython.core.display.Javascript object>